input_test, lowest location from range 46

In [1]:
import numpy as np
import re

In [2]:
#with open("input_test.txt", 'r') as f0:
with open("input.txt", 'r') as f0:
    text= f0.read()

In [3]:
class alm_map():
    def __init__(self, text_map):
        # text_map contains the text with lines providing map
        self.params_list=[]
        lines_map = text_map.split('\n')
        #print(lines_map)
        for l0 in lines_map:
            values_str = l0.split(' ')
            if len(values_str)!=3:
                #print(f"Error with this line {l0}, does not have 3 values")
                continue
            dest_orig = int(values_str[0])
            source_orig = int(values_str[1])
            length = int(values_str[2])
            self.params_list.append((dest_orig, source_orig, length))
    
    def source_to_dest(self,i):
        ret=i # default
        # go though list and finds the map
        for d0, s0, l0 in self.params_list:
            if i>=s0 and i<s0+l0:
                ret=d0+(i-s0)
                break
        return ret
    
    @staticmethod
    def _range_to_map_range(i0, i1, r0, r1, d):
        # put start end values in a list, convert to set to remove duplicates
        #range start, length i0,il. mapped range start, length r0 rl, destination start d
        
        start0 = max(i0,r0)
        end0 = min(i1,r1)

        ret_ranges_unmapped = [] #list of start,end
        ret_ranges_mapped = [] #list of start,end

        #Check region is inside the range and map it
        if start0<end0:
            ret_ranges_mapped.append((d+(start0-r0), d+(end0-r0)))

            #sort surrounding regions if any and mark them as unmmaped
            if start0>i0:
                ret_ranges_unmapped.append((i0,start0)) #start,end
            if i1>end0:
                ret_ranges_unmapped.append((end0,i1)) #start,end
        else:
            ret_ranges_unmapped.append((i0,i1))
            
        return [ret_ranges_unmapped,ret_ranges_mapped]
    
    def sources_to_dests_ranges(self, sources0):
        # use notation start,end
        #sources are a list of 2-tuples start:end
        unmapped_ranges0=sources0.copy() # initialize
        new_ranges=[]

        while len(unmapped_ranges0)>0:
            s0, s1 = unmapped_ranges0.pop()
            #print(f"origin range {s0}:{s1}")
            #Check against the mapping parameters
            new_unmapped = []
            new_mapped = []
            for d0, r0, l0 in self.params_list:
                r1=r0+l0 #enpoint
                #print(f"mapping r0:r1 {r0}:{r1} to d0:{d0}:{d0+l0}, be applied to unmapped_range:{s0}:{s1}")

                unmapped_ranges1, mapped_ranges1= self._range_to_map_range(s0,s1,r0,r1,d0)
                #print(f"result unmapped:{unmapped_ranges1}, mapped:{mapped_ranges1}")
                
                if len(mapped_ranges1)>0:
                    #print("len(mapped_ranges1)>0")
                    new_mapped+=mapped_ranges1 #mapped ranges increase
                    new_unmapped+=unmapped_ranges1
                    #Cannot use the original unmapped region, as it has alrady been transformed
                    # So leave the for-loop to get the next unmasked range
                    break

            # After checking an unmapped region against all the maps...
            # If there was no mapping applied, preserve it for next
            # If there was some mapping, then new_unmapped should contain the new maps
            # We need to check these new regions against the mappings
            if len(new_mapped)==0:
                #original range not affected
                #print(f"Region {s0}:{s1} not affected. new ranges+={(s0,s1)}")
                new_ranges+=[(s0,s1)]
            else:
                unmapped_ranges0+=new_unmapped
                new_ranges+=new_mapped
                #new_ranges+=new_unmapped

        return new_ranges

# Test _range_to_map_range

In [4]:
#Returns unmapped, mapped ranges, each in format start,length
print( alm_map._range_to_map_range(10,20, 5,15, 20) )

[[(15, 20)], [(25, 30)]]


Get starting seeds

In [5]:
seeds_inp = re.findall("seeds:(.*)", text)
seeds_inp

[' 202517468 131640971 1553776977 241828580 1435322022 100369067 2019100043 153706556 460203450 84630899 3766866638 114261107 1809826083 153144153 2797169753 177517156 2494032210 235157184 856311572 542740109']

convert to seed ranges

In [6]:
seeds_inp[0].strip().split()

['202517468',
 '131640971',
 '1553776977',
 '241828580',
 '1435322022',
 '100369067',
 '2019100043',
 '153706556',
 '460203450',
 '84630899',
 '3766866638',
 '114261107',
 '1809826083',
 '153144153',
 '2797169753',
 '177517156',
 '2494032210',
 '235157184',
 '856311572',
 '542740109']

In [7]:
seeds_ranges =[]

s_list = seeds_inp[0].strip().split()
for i in range(0,len(s_list),2):
    s0=int(s_list[i])
    s1=int(s_list[i+1]) +s0
    seeds_ranges.append((s0,s1))
seeds_ranges

[(202517468, 334158439),
 (1553776977, 1795605557),
 (1435322022, 1535691089),
 (2019100043, 2172806599),
 (460203450, 544834349),
 (3766866638, 3881127745),
 (1809826083, 1962970236),
 (2797169753, 2974686909),
 (2494032210, 2729189394),
 (856311572, 1399051681)]

In [8]:
# Overrides for testing
#seeds_ranges=[(82,1)] #test ok, gives right answer 46

In [9]:
# seed2soil
t0= re.findall("seed-to-soil map:\n((.|\n)*)\nsoil-to-fertilizer map", text)
t0_map = alm_map(t0[0][0])

In [10]:
start_ranges = seeds_ranges.copy()

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

soil0_ranges =result_ranges

new_range:[(1233103806, 1265974898), (1173097443, 1211564117), (2000660015, 2025282601), (476432861, 609280127), (1059486394, 1086566452), (943241642, 1059486394), (1632469937, 1787834568), (2044594803, 2059837853), (2343571960, 2407155294), (3398847262, 3540020132), (3381542286, 3398847262), (2796989323, 2810085327), (2976842391, 3029788837), (2457022734, 2581593444), (2025282601, 2044594803), (1086566452, 1116107747), (1287564557, 1357621124), (609280127, 612853360), (138376916, 169037772), (4289342532, 4294967296), (3949118541, 4013393462), (3540020132, 3584381554), (1815925015, 1900555914), (1172237554, 1173097443), (612853360, 657286135), (2064392707, 2172806599), (672016787, 772385854), (1357621124, 1393363309), (460283, 124156390), (1553049016, 1585337376), (0, 460283), (790471742, 840113387), (246837326, 378478297)]


In [11]:
soil0_ranges

[(1233103806, 1265974898),
 (1173097443, 1211564117),
 (2000660015, 2025282601),
 (476432861, 609280127),
 (1059486394, 1086566452),
 (943241642, 1059486394),
 (1632469937, 1787834568),
 (2044594803, 2059837853),
 (2343571960, 2407155294),
 (3398847262, 3540020132),
 (3381542286, 3398847262),
 (2796989323, 2810085327),
 (2976842391, 3029788837),
 (2457022734, 2581593444),
 (2025282601, 2044594803),
 (1086566452, 1116107747),
 (1287564557, 1357621124),
 (609280127, 612853360),
 (138376916, 169037772),
 (4289342532, 4294967296),
 (3949118541, 4013393462),
 (3540020132, 3584381554),
 (1815925015, 1900555914),
 (1172237554, 1173097443),
 (612853360, 657286135),
 (2064392707, 2172806599),
 (672016787, 772385854),
 (1357621124, 1393363309),
 (460283, 124156390),
 (1553049016, 1585337376),
 (0, 460283),
 (790471742, 840113387),
 (246837326, 378478297)]

soil-to-fertilizer map:

In [12]:
# soil2fert
t0 = re.findall("soil-to-fertilizer map:\n((.|\n)*)\nfertilizer-to-water map:", text)
t0_map = alm_map(t0[0][0])

# start_ranges = soil0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"soil_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range

# fert0_ranges=mapped_ranges

start_ranges = soil0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")
    
fert0_ranges =result_ranges #Adjust

new_range:[(2161962664, 2293603635), (1845319553, 1857803350), (1568121461, 1605279309), (1609507353, 1609967636), (110704112, 142992472), (1609967636, 1668507004), (1668507004, 1733663743), (524485951, 560228136), (1459294850, 1550035573), (1605279309, 1609507353), (2587142125, 2592542425), (983065601, 1004042197), (796954536, 884391832), (2527978698, 2572411473), (458556698, 459416587), (609209731, 654539235), (212737043, 234272290), (898322855, 916089003), (4215158378, 4259519800), (3090686214, 3122295925), (3827254494, 3859919704), (2904190668, 2909815432), (1747884269, 1778545125), (2524405465, 2527978698), (255693782, 271197184), (504622935, 524485951), (1136906676, 1143939070), (978403972, 983065601), (1113910550, 1136906676), (2104256415, 2133797710), (757844430, 777156632), (1323775159, 1448345869), (3454717424, 3468552021), (3483324631, 3522436480), (3274864356, 3287960360), (2691356327, 2708661303), (2708661303, 2726192707), (4091516912, 4215158378), (401354958, 419566719), 

In [13]:
fert0_ranges

[(2161962664, 2293603635),
 (1845319553, 1857803350),
 (1568121461, 1605279309),
 (1609507353, 1609967636),
 (110704112, 142992472),
 (1609967636, 1668507004),
 (1668507004, 1733663743),
 (524485951, 560228136),
 (1459294850, 1550035573),
 (1605279309, 1609507353),
 (2587142125, 2592542425),
 (983065601, 1004042197),
 (796954536, 884391832),
 (2527978698, 2572411473),
 (458556698, 459416587),
 (609209731, 654539235),
 (212737043, 234272290),
 (898322855, 916089003),
 (4215158378, 4259519800),
 (3090686214, 3122295925),
 (3827254494, 3859919704),
 (2904190668, 2909815432),
 (1747884269, 1778545125),
 (2524405465, 2527978698),
 (255693782, 271197184),
 (504622935, 524485951),
 (1136906676, 1143939070),
 (978403972, 983065601),
 (1113910550, 1136906676),
 (2104256415, 2133797710),
 (757844430, 777156632),
 (1323775159, 1448345869),
 (3454717424, 3468552021),
 (3483324631, 3522436480),
 (3274864356, 3287960360),
 (2691356327, 2708661303),
 (2708661303, 2726192707),
 (4091516912, 4215158378

In [14]:
# fert2water
t0 = re.findall("fertilizer-to-water map:\n((.|\n)*)\nwater-to-light map", text)
t0_map = alm_map(t0[0][0])

# start_ranges = fert0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range

# water0_ranges=mapped_ranges

start_ranges = fert0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

water0_ranges =result_ranges #Adjust

new_range:[(817911646, 850782738), (217878434, 256345108), (491683691, 516306277), (2425421388, 2431754666), (2369332991, 2399979338), (2453979874, 2549847515), (1835638204, 1862718262), (1719393452, 1835638204), (2241013061, 2263625071), (902400917, 972838569), (674550850, 736865819), (535618479, 550861529), (986997993, 1032369566), (159816805, 178028566), (3065354609, 3188996075), (3390047845, 3407579249), (3372742869, 3390047845), (3956250898, 3969346902), (4202282109, 4241393958), (4173674902, 4187509499), (1082237006, 1206807716), (516306277, 535618479), (1862718262, 1892259557), (872372397, 895368523), (736865819, 741527448), (895368523, 902400917), (263084782, 282947798), (14155629, 29659031), (2399979338, 2403552571), (1506346116, 1537006972), (3585577210, 3591201974), (2801092191, 2833757401), (3772072756, 3803682467), (3188996075, 3233357497), (656784702, 674550850), (2263625071, 2285160318), (367671578, 413001082), (217018545, 217878434), (2403552571, 2425421388), (413505296

In [15]:
water0_ranges

[(817911646, 850782738),
 (217878434, 256345108),
 (491683691, 516306277),
 (2425421388, 2431754666),
 (2369332991, 2399979338),
 (2453979874, 2549847515),
 (1835638204, 1862718262),
 (1719393452, 1835638204),
 (2241013061, 2263625071),
 (902400917, 972838569),
 (674550850, 736865819),
 (535618479, 550861529),
 (986997993, 1032369566),
 (159816805, 178028566),
 (3065354609, 3188996075),
 (3390047845, 3407579249),
 (3372742869, 3390047845),
 (3956250898, 3969346902),
 (4202282109, 4241393958),
 (4173674902, 4187509499),
 (1082237006, 1206807716),
 (516306277, 535618479),
 (1862718262, 1892259557),
 (872372397, 895368523),
 (736865819, 741527448),
 (895368523, 902400917),
 (263084782, 282947798),
 (14155629, 29659031),
 (2399979338, 2403552571),
 (1506346116, 1537006972),
 (3585577210, 3591201974),
 (2801092191, 2833757401),
 (3772072756, 3803682467),
 (3188996075, 3233357497),
 (656784702, 674550850),
 (2263625071, 2285160318),
 (367671578, 413001082),
 (217018545, 217878434),
 (2403552

In [16]:
# water2light
t0 = re.findall("water-to-light map:\n((.|\n)*)\nlight-to-temperature map", text)
t0_map = alm_map(t0[0][0])

# start_ranges = water0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# light0_ranges=mapped_ranges

start_ranges = water0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

light0_ranges =result_ranges #Adjust

new_range:[(3482802522, 3483979976), (3110800852, 3241264369), (2794157741, 2806641538), (2516959649, 2554117497), (2558345541, 2558805824), (3351968481, 3384256841), (2558805824, 2617345192), (2617345192, 2682501931), (329110061, 364852246), (2408133038, 2498873761), (2554117497, 2558345541), (1669461828, 1669738154), (1960328365, 1965452339), (741527448, 762504044), (0, 28193211), (622053693, 660822731), (601578646, 622053693), (1632167218, 1654731176), (3593928912, 3615797729), (263180808, 264040697), (413833841, 459163345), (3454001412, 3475536659), (42124234, 46162263), (660822731, 674550850), (1880519447, 1924880869), (1444763108, 1476372819), (4047584953, 4080250163), (1258267562, 1263892326), (2696722457, 2727383313), (3590355679, 3593928912), (60317892, 75821294), (309247045, 329110061), (895368523, 902400917), (736865819, 741527448), (872372397, 895368523), (3053094603, 3082635898), (562468540, 581780742), (2272613347, 2397184057), (1670789152, 1684623749), (1699396359, 17385

In [17]:
light0_ranges

[(3482802522, 3483979976),
 (3110800852, 3241264369),
 (2794157741, 2806641538),
 (2516959649, 2554117497),
 (2558345541, 2558805824),
 (3351968481, 3384256841),
 (2558805824, 2617345192),
 (2617345192, 2682501931),
 (329110061, 364852246),
 (2408133038, 2498873761),
 (2554117497, 2558345541),
 (1669461828, 1669738154),
 (1960328365, 1965452339),
 (741527448, 762504044),
 (0, 28193211),
 (622053693, 660822731),
 (601578646, 622053693),
 (1632167218, 1654731176),
 (3593928912, 3615797729),
 (263180808, 264040697),
 (413833841, 459163345),
 (3454001412, 3475536659),
 (42124234, 46162263),
 (660822731, 674550850),
 (1880519447, 1924880869),
 (1444763108, 1476372819),
 (4047584953, 4080250163),
 (1258267562, 1263892326),
 (2696722457, 2727383313),
 (3590355679, 3593928912),
 (60317892, 75821294),
 (309247045, 329110061),
 (895368523, 902400917),
 (736865819, 741527448),
 (872372397, 895368523),
 (3053094603, 3082635898),
 (562468540, 581780742),
 (2272613347, 2397184057),
 (1670789152, 168

In [18]:
# light2temp
t0 = re.findall("light-to-temperature map:\n((.|\n)*)\ntemperature-to-humidity map:", text)
t0_map = alm_map(t0[0][0])

# start_ranges = light0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# temp0_ranges=mapped_ranges
# temp0_ranges

start_ranges = light0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

temp0_ranges =result_ranges #Adjust

new_range:[(950677812, 983548904), (423611781, 462078455), (697417038, 722039624), (1749636588, 1755969866), (3993609472, 4008144721), (1708083440, 1724194538), (2649579616, 2655628421), (1778195074, 1868013910), (2008850096, 2022384013), (4281421155, 4294967296), (4165176403, 4281421155), (3469332789, 3470759652), (3429507993, 3450693140), (1035167083, 1105604735), (83402633, 145717602), (0, 5875575), (741351826, 750719301), (1119764159, 1125748475), (1903813542, 1943200799), (365550152, 383761913), (3627002669, 3750644135), (812010728, 829542132), (794705752, 812010728), (2212969206, 2226065210), (3569521047, 3608632896), (3540913840, 3554748437), (1993068239, 2008850096), (2435542069, 2458692982), (2672085239, 2757723179), (722039624, 729505231), (729505231, 741351826), (2022384013, 2051925308), (1005138563, 1028134689), (145717602, 150379231), (1028134689, 1035167083), (468818129, 488681145), (219888976, 235392378), (1724194538, 1727767771), (2379526415, 2410187271), (1258267562, 1

In [19]:
temp0_ranges

[(950677812, 983548904),
 (423611781, 462078455),
 (697417038, 722039624),
 (1749636588, 1755969866),
 (3993609472, 4008144721),
 (1708083440, 1724194538),
 (2649579616, 2655628421),
 (1778195074, 1868013910),
 (2008850096, 2022384013),
 (4281421155, 4294967296),
 (4165176403, 4281421155),
 (3469332789, 3470759652),
 (3429507993, 3450693140),
 (1035167083, 1105604735),
 (83402633, 145717602),
 (0, 5875575),
 (741351826, 750719301),
 (1119764159, 1125748475),
 (1903813542, 1943200799),
 (365550152, 383761913),
 (3627002669, 3750644135),
 (812010728, 829542132),
 (794705752, 812010728),
 (2212969206, 2226065210),
 (3569521047, 3608632896),
 (3540913840, 3554748437),
 (1993068239, 2008850096),
 (2435542069, 2458692982),
 (2672085239, 2757723179),
 (722039624, 729505231),
 (729505231, 741351826),
 (2022384013, 2051925308),
 (1005138563, 1028134689),
 (145717602, 150379231),
 (1028134689, 1035167083),
 (468818129, 488681145),
 (219888976, 235392378),
 (1724194538, 1727767771),
 (2379526415,

In [20]:
# temp2humid
t0 = re.findall("temperature-to-humidity map:\n((.|\n)*)\nhumidity-to-location map:", text)
t0_map = alm_map(t0[0][0])

# start_ranges = temp0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# humid0_ranges=mapped_ranges
# humid0_ranges

start_ranges = temp0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

humid0_ranges =result_ranges #Adjust

new_range:[(3972852039, 3974029493), (3274113751, 3285921340), (3840184189, 3887188384), (2961202681, 3032854414), (2928382729, 2940866526), (2005548599, 2042706447), (2046934491, 2047394774), (4197950728, 4208413467), (4176125107, 4197950728), (2047394774, 2076659641), (2603199191, 2632473692), (2632473692, 2658662447), (2313231287, 2352199271), (496607968, 532350153), (1941982137, 1987462711), (3896960389, 3942220538), (2042706447, 2046934491), (3490442665, 3490718991), (2286736616, 2291860590), (891412290, 903197033), (254282746, 263474599), (993228240, 1003776701), (967549573, 981933850), (263474599, 266735072), (1012372596, 1051141634), (241497467, 253376619), (1003776701, 1012372596), (1632167218, 1654731176), (1727767771, 1749636588), (430678715, 431538604), (581331748, 626661252), (3944050929, 3965586176), (161974660, 166012689), (1051141634, 1064869753), (2206927698, 2251289120), (1444763108, 1476372819), (2593382156, 2595247386), (3097031068, 3127831048), (1258267562, 1263892

In [21]:
humid0_ranges

[(3972852039, 3974029493),
 (3274113751, 3285921340),
 (3840184189, 3887188384),
 (2961202681, 3032854414),
 (2928382729, 2940866526),
 (2005548599, 2042706447),
 (2046934491, 2047394774),
 (4197950728, 4208413467),
 (4176125107, 4197950728),
 (2047394774, 2076659641),
 (2603199191, 2632473692),
 (2632473692, 2658662447),
 (2313231287, 2352199271),
 (496607968, 532350153),
 (1941982137, 1987462711),
 (3896960389, 3942220538),
 (2042706447, 2046934491),
 (3490442665, 3490718991),
 (2286736616, 2291860590),
 (891412290, 903197033),
 (254282746, 263474599),
 (993228240, 1003776701),
 (967549573, 981933850),
 (263474599, 266735072),
 (1012372596, 1051141634),
 (241497467, 253376619),
 (1003776701, 1012372596),
 (1632167218, 1654731176),
 (1727767771, 1749636588),
 (430678715, 431538604),
 (581331748, 626661252),
 (3944050929, 3965586176),
 (161974660, 166012689),
 (1051141634, 1064869753),
 (2206927698, 2251289120),
 (1444763108, 1476372819),
 (2593382156, 2595247386),
 (3097031068, 312783

In [22]:
# hum2loc
t0 = re.findall("humidity-to-location map:\n((.|\n)*)\n", text)
t0_map = alm_map(t0[0][0])

# start_ranges = humid0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# loc0_ranges=mapped_ranges
# loc0_ranges

start_ranges = humid0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

loc0_ranges =result_ranges #Adjust

new_range:[(825243414, 849169568), (958604635, 967549573), (319588639, 358055313), (602801739, 627424325), (2190963215, 2197296493), (1251652677, 1265177431), (3334546317, 3335556812), (2149410067, 2165521165), (3864664154, 3870712959), (1100176495, 1153197817), (2527540408, 2564337922), (3294324826, 3307858743), (3552519730, 3566065871), (2452799120, 2527540408), (3511016266, 3552519730), (3699524423, 3700951286), (1486358146, 1507543293), (2655090225, 2673394026), (1069388363, 1087300934), (156177746, 190399026), (1064869753, 1069388363), (37384986, 95181345), (789575380, 795450955), (646736527, 653782902), (220792608, 223113708), (2687553450, 2693537766), (1525779414, 1531539628), (2600137554, 2632545935), (3975108309, 3976326971), (190399026, 205587077), (714822659, 717846369), (3935008370, 3975108309), (2959006122, 3022029564), (3911390684, 3931908769), (284405135, 301936539), (267100159, 284405135), (1966643921, 1979739925), (2293801695, 2299287259), (3732263270, 3747294236), (13

In [23]:
loc0_ranges

[(825243414, 849169568),
 (958604635, 967549573),
 (319588639, 358055313),
 (602801739, 627424325),
 (2190963215, 2197296493),
 (1251652677, 1265177431),
 (3334546317, 3335556812),
 (2149410067, 2165521165),
 (3864664154, 3870712959),
 (1100176495, 1153197817),
 (2527540408, 2564337922),
 (3294324826, 3307858743),
 (3552519730, 3566065871),
 (2452799120, 2527540408),
 (3511016266, 3552519730),
 (3699524423, 3700951286),
 (1486358146, 1507543293),
 (2655090225, 2673394026),
 (1069388363, 1087300934),
 (156177746, 190399026),
 (1064869753, 1069388363),
 (37384986, 95181345),
 (789575380, 795450955),
 (646736527, 653782902),
 (220792608, 223113708),
 (2687553450, 2693537766),
 (1525779414, 1531539628),
 (2600137554, 2632545935),
 (3975108309, 3976326971),
 (190399026, 205587077),
 (714822659, 717846369),
 (3935008370, 3975108309),
 (2959006122, 3022029564),
 (3911390684, 3931908769),
 (284405135, 301936539),
 (267100159, 284405135),
 (1966643921, 1979739925),
 (2293801695, 2299287259),
 (

In [24]:
# lowest number
loc_starts=[]
for loc0 in loc0_ranges:
    s0,l0 = loc0
    loc_starts.append(s0)

print(np.min(loc_starts))

37384986


Answer 149145352 ? is too high!!

Correct answer is 37384986